widget:- especially when building graphical user interfaces (GUIs) or interactive applications, a widget is a UI (User Interface) element like:
Button
Text box
Dropdown menu
Slider
Label
Checkbox
Widgets are used to interact with users — get input, display output, or perform actions.

In [ ]:
#Manual S3 UPLOAD
from IPython.display import display
import ipywidgets as widgets
import tempfile

upload_widget = widgets.FileUpload(
    accept='.mp3,.wav,.flac',
    multiple=False
)

display(upload_widget)


FileUpload(value=(), accept='.mp3,.wav,.flac', description='Upload')

complete FileUpload(value=(), accept='.mp3,.wav,.flac', description='Upload')


In [3]:
print("complete",upload_widget)

complete FileUpload(value=({'name': 'repeat-the-sentence-.wav', 'type': 'audio/wav', 'size': 3899436, 'content': <memory at 0x1079cdf40>, 'last_modified': datetime.datetime(2025, 7, 8, 6, 38, 34, 850000, tzinfo=datetime.timezone.utc)},), accept='.mp3,.wav,.flac', description='Upload')


 What this code does:
Creates a temp file with a unique name (ending in _filename).
Writes data (content) to that file.
Keeps the file after the block ends (because of delete=False).
Stores the file path in local_path.

In [ ]:
# Save the uploaded file
if upload_widget.value:
    # Since upload_widget.value is a tuple, get the first item
    file_info = upload_widget.value[0]
    
    filename = file_info['name']
    content = file_info['content']

    import tempfile
    with tempfile.NamedTemporaryFile(delete=False, suffix='_' + filename) as f:
        f.write(content)
        local_path = f.name

    print(f"✅ File saved to: {local_path}")
else:
    print("⚠️ Please upload a file first.")

In [ ]:
import boto3

s3 = boto3.client('s3')
bucket_name = 'audiototextproject9559'  # Replace with your bucket
s3_key = filename

# Upload to S3
s3.upload_file(local_path, bucket_name, s3_key)
print(f"Uploaded to S3: s3://{bucket_name}/{s3_key}")

In [ ]:
#starting from here directly from the uploaded file
#DIRECT S3 UPLOAD

In [4]:
#here
import boto3
import time
import json
import pandas as pd
from urllib.request import urlopen

# -------------------------------
# STEP 1: Transcribe Audio via Amazon Transcribe
# -------------------------------

transcribe = boto3.client("transcribe", region_name="us-east-1")
s3_audio_uri = "s3://audiototextproject9559/AUDIO_TEXT/archive/call_recording_02.wav"  # Replace with your actual S3 URI
job_name = "JOB1"

# Start transcription
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': s3_audio_uri},
    MediaFormat='wav',
    LanguageCode='en-US'
)

# Wait for transcription to complete
while True:
    result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    status = result['TranscriptionJob']['TranscriptionJobStatus']
    if status in ['COMPLETED', 'FAILED']:
        break
    print("⏳ Waiting for transcription to complete...")
    time.sleep(5)

# Load the transcript
transcript_uri = result['TranscriptionJob']['Transcript']['TranscriptFileUri']
transcribed_text = json.loads(urlopen(transcript_uri).read())['results']['transcripts'][0]['transcript']
print("📝 Transcribed Text:\n", transcribed_text)

⏳ Waiting for transcription to complete...
⏳ Waiting for transcription to complete...
📝 Transcribed Text:
 I am extremely dissatisfied with my recent order. I am John Davis and I'm calling about order number 123456. The FR 4401 refrigerator arrived yesterday, and it's damaged. There's a huge dent on the side, and it's not cooling properly. I want to know what you're going to do about this. I demand the replacement, and I want it delivered as soon as possible. I've been without a working refrigerator for 2 days now.


In [5]:
transcript_uri

'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/387529818903/JOB1/6c6281ab-8e04-4978-8c2a-2c0ea0d4ee01/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIBPlY1wAEFKTK9ajNAOwbTlq0LLw7wOKL%2F7hXzEtXB23AiEAnkPyTJR2YfOxbfSW8pbQnDk5Qb6LlYQ4xpqaYRgISzgquwUIpP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAEGgwyNzY2NTY0MzMxNTMiDLYf1a0BDp0x6Q5qTSqPBWbCcTARGOtpb8G2JhR3WXWVj8I3r1HSwPaQCDPru3Aq0Ar0Wg7Ca5zNc8Qy6DSMcrSzQjun2PBOAOi0NTOZh1JARelAmrnpFgcLY0QrdiPJRLcDkqxSnGk5aF7XfX26pcgcVzry6vL3L84hnOrnD2piOQsNtsKcGu7v46NMvcLOMMAY6UF6TN68T4A4BAjGqCv%2B7w%2FG2KGWB%2FxcH0L%2BoU3mCDLBlPrposiG7NdqtuFb0ROrk%2BfCEFRKN%2FtuLM6NXCdW%2ByE%2FgnVaIeYjkGM1z33ok8l1Pja0D%2BPFOdtWCQV3Uef7jaxz2iqw7EJnT0FfxnT%2F6K6iZqRo9mpcEaj%2B42lX8DxAnESQ7blcN8yq1HD2obr43iCjJxAlnlRjYyp%2BSqXWdcF6glmQcuaBqpbEqRN%2FaMMUJY8YRMn6x5kHYSiZu7fLL6%2FBLrz5tmwUpBcThYnKHXj4n8U%2B9w6Dno%2F%2BMJce27K5duU2%2FU717VaudFNeRuVVj7QPoFyl8Zb4BV1PgJ3wZckgnr%2FDeSynFD%2FzmLEpWUG53sGtQJUN1S%2FUUAa

In [6]:


# -------------------------------
# STEP 2: Send Transcribed Text to Claude 3 Sonnet (Bedrock)
# -------------------------------

bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

messages = [
    {
        "role": "user",
        "content": f"""Please extract the following structured information from the call transcript below.

Transcript:
{transcribed_text}

Return the output in JSON format with these keys:
- unique_id
- call_type (product enquiry, complaint, order, tech issue, etc.)
- sentiment
- customer_name
- order_number
- product_number
- description
"""
    }
]

response = bedrock.invoke_model(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps({
        "messages": messages,
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 800,
        "temperature": 0.3
    })
)

# Parse response
output = json.loads(response['body'].read())
extracted_json = json.loads(output["content"][0]["text"])




In [7]:
# -------------------------------
# STEP 3: Convert to Pandas DataFrame
# -------------------------------
df = pd.DataFrame([extracted_json])
print("📊 Structured Output as DataFrame:")
print(df)


📊 Structured Output as DataFrame:
  unique_id  call_type               sentiment customer_name order_number  \
0         1  complaint  extremely dissatisfied    John Davis       123456   

  product_number                                        description  
0        FR 4401  The refrigerator arrived damaged with a huge d...  


In [8]:
import pandas as pd

# Step 1: Add flag based on conditions
def get_flag(row):
    if row['call_type'].lower() == "complaint" or row['sentiment'].lower() in ["frustrated", "negative", "angry"]:
        return "Red"
    return "Green"

df["flag"] = df.apply(get_flag, axis=1)

# Step 2: Save CSV

df.to_csv('csv_file_path.csv', index=False)

In [9]:
import boto3
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText

# SES setup
ses = boto3.client('ses', region_name='us-east-1')

SENDER = "ashukhodwalalwar1@gmail.com"     # Must be verified in SES
RECIPIENT = "ashukhodwalalwar1@gmail.com"             # Also verified in SES sandbox mode
SUBJECT = "Update-9th july-📞 Call Summary Report with Risk Flag"

# Create the email
msg = MIMEMultipart()
msg['Subject'] = SUBJECT
msg['From'] = SENDER
msg['To'] = RECIPIENT

# Body
body = MIMEText("Please find attached the CSV file with call summary and risk flags.", "plain")
msg.attach(body)

# Attach CSV
with open("csv_file_path.csv", 'rb') as f:
    part = MIMEApplication(f.read(), Name="call_summary.csv")
    part['Content-Disposition'] = 'attachment; filename="call_summary.csv"'
    msg.attach(part)

# Send email
response = ses.send_raw_email(
    Source=SENDER,
    Destinations=[RECIPIENT],
    RawMessage={'Data': msg.as_string()}
)

print("📤 Email sent via SES! Message ID:", response['MessageId'])


📤 Email sent via SES! Message ID: 01000197ef12b7e2-90108d7a-a3e4-46f1-8352-474a3f223673-000000
